#### Project Idea : News Headlines (Canada, World, Business, Technology, Entertainment, Sports, Science, Health) ######

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

import json 
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.decomposition import PCA 
from sklearn.cluster import KMeans 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,mean_squared_error
from sklearn.metrics import silhouette_score

#### STEP 1. IMPORT URLS & CONTENT AND GET THE HEADLINES ####
8 MAJOR SECTIONS OF GOOGLE NEWS: Canada, World, Business, Technology, Entertainment, Sports, Science, Health

In [9]:
def scrape_headlines_from_urls(url_list):
    all_headlines = []

    for url in url_list:
        # Fetch content from URL
        response = requests.get(url)
        print(response.status_code)
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the content with BeautifulSoup
            soup = bs(response.text, 'html.parser')
            # Find h4 elements that might contain headlines
            headlines = soup.find_all('a',class_='gPFEn')
            print(headlines)
            # Extract the text from each headline and add to the list
            for headline in headlines:
                all_headlines.append(headline.text.strip())  # .strip() to remove leading/trailing whitespaces
        else:
            print(f'Failed to retrieve {url} with status code: {response.status_code}')

    return all_headlines

# Example usage:
urls = [
    #Canada
    'https://news.google.com/topics/CAAqJggKIiBDQkFTRWdvSkwyMHZNR1F3TmpCbkVnVmxiaTFIUWlnQVAB?hl=en-CA&gl=CA&ceid=CA%3Aen',
    #World: 
    'https://news.google.com/topics/CAAqKggKIiRDQkFTRlFvSUwyMHZNRGx1YlY4U0JXVnVMVWRDR2dKRFFTZ0FQAQ?hl=en-CA&gl=CA&ceid=CA%3Aen',
    #Business
    'https://news.google.com/topics/CAAqKggKIiRDQkFTRlFvSUwyMHZNRGx6TVdZU0JXVnVMVWRDR2dKRFFTZ0FQAQ?hl=en-CA&gl=CA&ceid=CA%3Aen',
    #Technology
    'https://news.google.com/topics/CAAqKggKIiRDQkFTRlFvSUwyMHZNRGRqTVhZU0JXVnVMVWRDR2dKRFFTZ0FQAQ?hl=en-CA&gl=CA&ceid=CA%3Aen',
    #Entertainment
    'https://news.google.com/topics/CAAqKggKIiRDQkFTRlFvSUwyMHZNREpxYW5RU0JXVnVMVWRDR2dKRFFTZ0FQAQ?hl=en-CA&gl=CA&ceid=CA%3Aen'
    #Sports
    'https://news.google.com/topics/CAAqKggKIiRDQkFTRlFvSUwyMHZNRFp1ZEdvU0JXVnVMVWRDR2dKRFFTZ0FQAQ?hl=en-CA&gl=CA&ceid=CA%3Aen',
    #Science
    'https://news.google.com/topics/CAAqKggKIiRDQkFTRlFvSUwyMHZNRFp0Y1RjU0JXVnVMVWRDR2dKRFFTZ0FQAQ/sections/CAQiSkNCQVNNUW9JTDIwdk1EWnRjVGNTQldWdUxVZENHZ0pEUVNJT0NBUWFDZ29JTDIwdk1ETTJYeklxQ2hJSUwyMHZNRE0yWHpJb0FBKi4IACoqCAoiJENCQVNGUW9JTDIwdk1EWnRjVGNTQldWdUxVZENHZ0pEUVNnQVABUAE?hl=en-CA&gl=CA&ceid=CA%3Aen',
    # Health
    'https://news.google.com/topics/CAAqJQgKIh9DQkFTRVFvSUwyMHZNR3QwTlRFU0JXVnVMVWRDS0FBUAE?hl=en-CA&gl=CA&ceid=CA%3Aen'
]

# Call the function and print the results
headlines = scrape_headlines_from_urls(urls)
for headline in headlines:
    print(headline)
len(headlines)

200
[<a class="gPFEn" data-n-tid="29" href="./articles/CBMicWh0dHBzOi8vbW9udHJlYWxnYXpldHRlLmNvbS9uZXdzL2xvY2FsLW5ld3MvbW9udHJlYWxzLW11Y2gtaGVyYWxkZWQtd2ludGVyLXN0b3JtLXF1aWNrbHktdHVybmVkLXRvLXJhaW4tb3Zlcm5pZ2h00gEA?hl=en-CA&amp;gl=CA&amp;ceid=CA%3Aen" jsaction="click:kkIcoc;" tabindex="0" target="_blank">Montreal storm update: Snow-clearing operations will begin Wednesday evening</a>, <a class="gPFEn" data-n-tid="29" href="./articles/CBMijQFodHRwczovL3d3dy5jdHZuZXdzLmNhL2NsaW1hdGUtYW5kLWVudmlyb25tZW50L2Zyb20td2luZC1jaGlsbHMtb2YtNTUtdG8tNDAtY20tb2Ytc25vdy13ZWF0aGVyLXdhcm5pbmdzLWluLXBsYWNlLWZvci1tdWNoLW9mLWNhbmFkYS0xLjY3MTkwMznSAQA?hl=en-CA&amp;gl=CA&amp;ceid=CA%3Aen" jsaction="click:kkIcoc;" tabindex="0" target="_blank">Canada weather forecast: Heavy snow, cold temperatures</a>, <a class="gPFEn" data-n-tid="29" href="./articles/CCAiC2JPY2ZZU1hJdm53mAEB?hl=en-CA&amp;gl=CA&amp;ceid=CA%3Aen" jsaction="click:kkIcoc;" tabindex="0" target="_blank">Major winter storm systems move across Canad

200
[<a class="gPFEn" data-n-tid="29" href="./articles/CBMiigFodHRwczovL3d3dy5jdHZuZXdzLmNhL3dvcmxkL2VjdWFkb3Itcy1lc2NhbGF0aW5nLWdhbmctdmlvbGVuY2UtaXMtYnJvYWRjYXN0LWxpdmUtdG8tdGhlLW5hdGlvbi1hcy1tYXNrZWQtZ3VubWVuLXN0b3JtLXR2LXN0dWRpby0xLjY3MTkzMDPSAQA?hl=en-CA&amp;gl=CA&amp;ceid=CA%3Aen" jsaction="click:kkIcoc;" tabindex="0" target="_blank">Ecuador news: Country placed under state of emergency</a>, <a class="gPFEn" data-n-tid="29" href="./articles/CBMiaWh0dHBzOi8vd3d3LmV1cm9uZXdzLmNvbS8yMDI0LzAxLzEwL2VjdWFkb3ItbGFiZWxzLTIwLWRydWctZ2FuZ3MtYXMtdGVycm9yaXN0LWdyb3Vwcy1hZnRlci1saXZlLXR2LWF0dGFja9IBAA?hl=en-CA&amp;gl=CA&amp;ceid=CA%3Aen" jsaction="click:kkIcoc;" tabindex="0" target="_blank">Ecuador deems drug gangs 'terrorist groups' after live TV attack</a>, <a class="gPFEn" data-n-tid="29" href="./articles/CBMiQGh0dHBzOi8vZ2xvYmFsbmV3cy5jYS9uZXdzLzEwMjE0MjQ4L2VjdWFkb3ItdHYtc3RhdGlvbi12aW9sZW5jZS_SAQA?hl=en-CA&amp;gl=CA&amp;ceid=CA%3Aen" jsaction="click:kkIcoc;" tabindex="0" target="_blank">

200
[<a class="gPFEn" data-n-tid="29" href="./articles/CBMiUmh0dHBzOi8vY2EuZmluYW5jZS55YWhvby5jb20vbmV3cy9iaXRjb2luLWV0Zi1ob3BlZnVscy1zdGlsbC1leHBlY3QtMTEwMzM5MjgxLmh0bWzSAQA?hl=en-CA&amp;gl=CA&amp;ceid=CA%3Aen" jsaction="click:kkIcoc;" tabindex="0" target="_blank">Bitcoin ETF hopefuls still expect SEC approval despite social media hack</a>, <a class="gPFEn" data-n-tid="29" href="./articles/CBMiSGh0dHBzOi8vd3d3LmNiYy5jYS9uZXdzL3dvcmxkL3VzLXNlYy1iaXRjb2luLWV0ZnMtbm90LWFwcHJvdmVkLTEuNzA3ODk5MdIBIGh0dHBzOi8vd3d3LmNiYy5jYS9hbXAvMS43MDc4OTkx?hl=en-CA&amp;gl=CA&amp;ceid=CA%3Aen" jsaction="click:kkIcoc;" tabindex="0" target="_blank">U.S. SEC has not approved bitcoin ETFs, says social media account was compromised</a>, <a class="gPFEn" data-n-tid="29" href="./articles/CBMiigFodHRwczovL3d3dy5mb3JleGxpdmUuY29tL0NyeXB0b2N1cnJlbmN5L3R3aWl0dGVyLWNvbmZpcm1zLXNlY2dvdi13YXMtY29tcHJvbWlzZWQtZGlkLW5vdC1oYXZlLXR3by1mYWN0b3ItYXV0aGVudGljYXRpb24tZW5hYmxlZC0yMDI0MDExMC_SAY4BaHR0cHM6Ly93d3cuZm9yZXhsaXZlLmNvb

200
[<a class="gPFEn" data-n-tid="29" href="./articles/CBMiYGh0dHBzOi8vd3d3LnV0aWxpdHlkaXZlLmNvbS9uZXdzL3BubmwtbWljcm9zb2Z0LWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLXJlc2VhcmNoLWJhdHRlcnkvNzA0MTc2L9IBAA?hl=en-CA&amp;gl=CA&amp;ceid=CA%3Aen" jsaction="click:kkIcoc;" tabindex="0" target="_blank">DOE lab, Microsoft find new battery material in AI-based energy storage research initiative</a>, <a class="gPFEn" data-n-tid="29" href="./articles/CBMiLGh0dHBzOi8vd3d3LmJiYy5jb20vbmV3cy90ZWNobm9sb2d5LTY3OTEyMDMz0gEwaHR0cHM6Ly93d3cuYmJjLmNvbS9uZXdzL3RlY2hub2xvZ3ktNjc5MTIwMzMuYW1w?hl=en-CA&amp;gl=CA&amp;ceid=CA%3Aen" jsaction="click:kkIcoc;" tabindex="0" target="_blank">New material found by AI could reduce lithium use in batteries</a>, <a class="gPFEn" data-n-tid="29" href="./articles/CCAiC29XYUxUWFRTUl9BmAEB?hl=en-CA&amp;gl=CA&amp;ceid=CA%3Aen" jsaction="click:kkIcoc;" tabindex="0" target="_blank">Microsoft Azure Quantum and PNNL join forces to accelerate scientific discovery</a>, <a class="gPFEn" data-n-t

200
[<a class="gPFEn" data-n-tid="29" href="./articles/CBMifWh0dHBzOi8vd3d3LnRoZWdsb2JlYW5kbWFpbC5jb20vYXJ0cy9maWxtL3Jldmlld3MvYXJ0aWNsZS1iaWJsaWNhbC1zdG9uZXItY29tZWR5LXRoZS1ib29rLW9mLWNsYXJlbmNlLXN1ZmZlcnMtZnJvbS1jcmlzaXMtb2Yv0gEA?hl=en-CA&amp;gl=CA&amp;ceid=CA%3Aen" jsaction="click:kkIcoc;" tabindex="0" target="_blank">Review: Biblical stoner comedy The Book of Clarence suffers from crisis of faith</a>, <a class="gPFEn" data-n-tid="29" href="./articles/CBMiSGh0dHBzOi8vd3d3LnJvZ2VyZWJlcnQuY29tL3Jldmlld3MvdGhlLWJvb2stb2YtY2xhcmVuY2UtZmlsbS1yZXZpZXctMjAyNNIBAA?hl=en-CA&amp;gl=CA&amp;ceid=CA%3Aen" jsaction="click:kkIcoc;" tabindex="0" target="_blank">The Book of Clarence movie review (2024)</a>, <a class="gPFEn" data-n-tid="29" href="./articles/CBMiP2h0dHBzOi8vd3d3LmxldmVsbWFuLmNvbS9qZXltZXMtc2FtdWVsLWJvb2stY2xhcmVuY2UtaW50ZXJ2aWV3L9IBAA?hl=en-CA&amp;gl=CA&amp;ceid=CA%3Aen" jsaction="click:kkIcoc;" tabindex="0" target="_blank">How Jeymes Samuel Made a Black Biblical Classic</a>, <a class

200
[<a class="gPFEn" data-n-tid="29" href="./articles/CBMigAFodHRwczovL3Rvcm9udG8uY2l0eW5ld3MuY2EvMjAyNC8wMS8xMC9hbmNpZW50LWh1bWFuLWRuYS1oaW50cy1hdC13aHktYS1tb2Rlcm4tZGlzZWFzZS1hZmZlY3RzLXNvLW1hbnktbm9ydGhlcm4tZXVyb3BlYW5zLXRvZGF5L9IBAA?hl=en-CA&amp;gl=CA&amp;ceid=CA%3Aen" jsaction="click:kkIcoc;" tabindex="0" target="_blank">Ancient human DNA hints at why multiple sclerosis affects so many northern Europeans today</a>, <a class="gPFEn" data-n-tid="29" href="./articles/CBMiY2h0dHBzOi8vd3d3Lndhc2hpbmd0b25wb3N0LmNvbS9zY2llbmNlLzIwMjQvMDEvMTAvYW5jaWVudC1kbmEtbXVsdGlwbGUtc2NsZXJvc2lzLWV1cm9wZWFuLWFuY2VzdHJ5L9IBAA?hl=en-CA&amp;gl=CA&amp;ceid=CA%3Aen" jsaction="click:kkIcoc;" tabindex="0" target="_blank">Ancient DNA helps trace multiple sclerosis origins in European descendants</a>, <a class="gPFEn" data-n-tid="29" href="./articles/CBMiKGh0dHBzOi8vd3d3LmJiYy5jb20vbmV3cy9oZWFsdGgtNjc5MTcyOTTSAQA?hl=en-CA&amp;gl=CA&amp;ceid=CA%3Aen" jsaction="click:kkIcoc;" tabindex="0" target="_blank">Scient

1070

#### STEP 2. CREATE DATAFRAME & FREEZE THE DATA GENERATED (CSV FILE) ####

In [10]:
df_=pd.DataFrame(headlines)
df_.columns=["Headline"]
df_.shape

#df_.to_csv("newsheadlines_11062023.csv")
#df_.to_csv("newsheadlines_11072023.csv")

(1070, 1)

In [11]:
csv_files=["newsheadlines_11062023.csv","newsheadlines_11072023.csv"]

df_csv_append = pd.DataFrame()

for file in csv_files:
    df=pd.read_csv(file)
    df_csv_append=df_csv_append.append(df, ignore_index=True)
    
df=df_csv_append
df #2349 headlines

FileNotFoundError: [Errno 2] No such file or directory: 'newsheadlines_11062023.csv'

#### STEP 3. CLEAN DATAFRAME  ####

In [ ]:
# Remove duplicates
df=df.drop_duplicates(subset = "Headline", keep = 'first')
df #reduced to 2016

# Remove \n
df.loc[:,"Headline Cleaned"]= df["Headline"].replace(r'\n',' ', regex=True) 

# Remove Punctuation
df.loc[:,"Headline Cleaned"]=df["Headline Cleaned"].str.replace(r'[^\w\s]+', '')

# Remove Capital Letters
df["Headline Cleaned"]=df["Headline Cleaned"].str.lower()

# Remove Stop Words
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stopwords=(stopwords.words('english'))
stopwords

df["Headline Cleaned"]=df["Headline Cleaned"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

# Remove Numbers
df["Headline Cleaned"]=df["Headline Cleaned"].str.replace('\d+', '',regex=True)


# Lemmatizing
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
lemmatized_string=[]

for row in range(0,len(df["Headline Cleaned"])):
    tokens=nltk.word_tokenize(df["Headline Cleaned"].iloc[row])
    lemmatized_string.append( ' '.join([lemmatizer.lemmatize(words) for words in tokens]))
    
df["Headline Cleaned"]=lemmatized_string

#Stemming
from nltk.stem import PorterStemmer, WordNetLemmatizer

stemmer = PorterStemmer()
df['Headline Cleaned'] = df['Headline Cleaned'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
df['Headline'] = df['Headline'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

df

#### STEP 4. VECTORIZATION ####

In [ ]:
tf=TfidfVectorizer()
X=df['Headline Cleaned']
X_=tf.fit_transform(X)
X_

words=tf.get_feature_names_out() #word associated to each vector
len(words) #3327 (without lemmatization and stemming, 4048)
# for word in words:
#      print(word)

#### STEP 5. CLUSTERING ####

In [ ]:
#Determining optimal number of clusters

# Calculate WCSS for a range of k values
WCSS = []
silhouette_scores = []
K_range = range(2, 15)  # You can choose the max value of k based on your dataset and needs

for k in K_range:
    #Fit a kmeans model to X_:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init = 10)
    kmeans.fit(X_)
    #Calculate silhouette score
    silhouette_average = silhouette_score(X_, kmeans.labels_)
    silhouette_scores.append(silhouette_average)
    # Inertia method returns WCSS for that model
    WCSS.append(kmeans.inertia_)

# Plot for WCSS
plt.figure(figsize=(10, 5))
plt.plot(K_range, WCSS, 'bo-', markerfacecolor='red')
plt.title('Elbow Method For Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Within-Cluster Sum of Squares (WCSS)')
plt.xticks(K_range)
plt.grid(True)
plt.show()  # This shows the first plot for WCSS

# Plot for silhouette scores
plt.figure(figsize=(10, 5))
plt.plot(K_range, silhouette_scores, 'bo-', markerfacecolor='blue')
plt.title('Silhouette Scores For Different k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.xticks(K_range)
plt.grid(True)
plt.show()  # This shows the second plot for silhouette scores

In [ ]:
#Kmean clustering: 

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Determine the number of clusters (e.g., using the elbow method)
k=8

# Apply KMeans clustering
kmeans = KMeans(n_clusters=k, random_state=42)
clusters = kmeans.fit_predict(X_)

# Dimensionality reduction for visualization
pca = PCA(n_components=2)
reduced_features = pca.fit_transform(X_.toarray())

# Plot the clusters
plt.figure(figsize=(10, 5))
plt.scatter(reduced_features[:, 0], reduced_features[:, 1], c=clusters, cmap='viridis', marker='o')
plt.title('KMeans Clustering Visualization')
plt.xlabel('PCA Feature 1')
plt.ylabel('PCA Feature 2')

# Plot the centroids
centroids = pca.transform(kmeans.cluster_centers_)
plt.scatter(centroids[:, 0], centroids[:, 1], marker='x', s=150, c='red')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
import numpy as np

# Create a new plot
plt.figure(figsize=(10, 5))

colors = plt.cm.tab10(np.arange(k))  # Generate color array for each cluster

# Plot each cluster
for i in range(k):
    # Select the indices of points in the current cluster
    indices = np.where(clusters == i)
    #print(indices)
    
    # Select the points that form the cluster
    cluster_points = reduced_features[indices]
    
    # Plot the points
    plt.scatter(cluster_points[:, 0], cluster_points[:, 1], c=[colors[i]], label=f'Cluster {i}')
    
    # Create the convex hull
    if len(cluster_points) > 2:  # ConvexHull can't be created with fewer than 3 points
        hull = ConvexHull(cluster_points)
        
        # Draw the convex hull
        # Generate the coordinates for the hull points
        hull_points = cluster_points[hull.vertices]
        
        # Fill the convex hull with a semi-opaque color and no border
        plt.fill(hull_points[:,0], hull_points[:,1], alpha=0.3, c=colors[i], edgecolor='none')

#Plot the centroids
centroids = pca.transform(kmeans.cluster_centers_)
plt.scatter(centroids[:, 0], centroids[:, 1], marker='x', s=150, c='red', label='Centroids')

# Final plot adjustments
plt.title('KMeans Clustering with Convex Hulls')
plt.xlabel('PCA Feature 1')
plt.ylabel('PCA Feature 2')
plt.legend()
plt.show()

In [ ]:
common_words = kmeans.cluster_centers_.argsort()[:,-1:-21:-1] #converts each centroid to a list of the columns most highly-valued words
for index, centroid in enumerate(common_words):
    print(str(index) + ' : ' + ', '.join(words[word] for word in centroid))

In [ ]:
#Plot the words

import matplotlib.pyplot as plt
   
x_axis=reduced_features[:, 0]
y_axis=reduced_features[:, 1]

fig,plt=plt.subplots(figsize=(20,20))

plt.scatter(reduced_features[:, 0], reduced_features[:, 1], c=clusters, cmap='viridis', marker='o')

words=tf.get_feature_names_out() #Get output feature names for transformation

for index,word in enumerate(words):
    plt.annotate(word,(x_axis[index],y_axis[index]))
    #print(word,(x_axis[index],y_axis[index]))

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

# Let's assume you have found an optimal number of clusters to be 'k'
k =8  # Replace with the number you determined

# Assuming X is your dataset
pca = PCA(n_components=3)
reduced_features = pca.fit_transform(X_.toarray())

# Apply KMeans clustering to the reduced data
kmeans = KMeans(n_clusters=k, random_state=42)
clusters = kmeans.fit_predict(reduced_features)

# Create a new plot for 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

colors = plt.cm.tab10(np.arange(10))  # Adjust range according to the number of clusters

# Plot each cluster in 3D
for i in range(k):
    # Select the indices of points in the current cluster
    indices = np.where(clusters == i)
    
    # Select the points that form the cluster
    cluster_points = reduced_features[indices]
    
    # Plot the points
    ax.scatter(cluster_points[:, 0], cluster_points[:, 1], cluster_points[:, 2], c=[colors[i]], label=f'Cluster {i}')

# Plotting centroids in 3D
centroids = pca.transform(kmeans.cluster_centers_)
ax.scatter(centroids[:, 0], centroids[:, 1], centroids[:, 2], marker='x', s=150, c='red', label='Centroids')

# Final plot adjustments
ax.set_title('3D KMeans Clustering')
ax.set_xlabel('PCA Feature 1')
ax.set_ylabel('PCA Feature 2')
ax.set_zlabel('PCA Feature 3')
plt.legend()
plt.show()